In [1]:
import torch

import transformers
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [29]:
def freq_dist(frame, colName):
    frows = frame.shape[0]
    freq = pd.DataFrame({'cnt': frame[colName].value_counts()})
    freq['percent'] = 100 * freq.cnt/frows
    return freq


In [22]:
def get_classes(row, labels, min_score):
    s = row[labels]
    s = s.sort_values(ascending=False)
    return s[s > min_score].index.values    

In [23]:
def results_to_df(results, labels, sequences, min_score = 0.9):
    nresults = len(results)
    scores = pd.DataFrame(0, index=range(nresults), columns=labels, dtype=float)

    nlabels = len(labels)
    for j in range(nresults):    
        r = results[j]

        for i in range(nlabels):
            scores.loc[j, r['labels'][i]] = r['scores'][i]

    scores['classes'] = scores.apply(lambda s: get_classes(s, labels, min_score), axis=1)
    scores['has_class'] = scores.classes.apply(lambda x: len(x) > 0)
    scores['max_class'] = scores.classes.apply(lambda x: x[0] if len(x) > 0 else None)
    scores['seq'] = sequences
    return scores

In [4]:
cols = open("videostoingest_cols.txt").readlines()
cols = [c.strip() for c in cols]
print(cols)

['VideoKey', 'VideoId', 'VideoUrl', 'ChannelName', 'ChannelId', 'Description', 'Title', 'PlayerType', 'PlayerParams', 'ProductsJson', 'ChannelImage', 'Status', 'StartTimeInMillis', 'DurationInSeconds', 'ThumbnailUrl', 'TrailerUrl', 'Tags', 'MinPrice', 'ViewCount', 'Market', 'ChannelKey', 'PartnerAlias', 'StreamingType', 'StreamingUrl']


In [5]:
vd = pd.read_csv('VideosToIngest_2022-09-01.tsv', sep='\t', header=None, names=cols)

In [6]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [15]:
candidate_labels = ['travel', 'food', 'kitchen', 'home improvement',  'home decor', 'home furnishings', 'apparel', 'shoes', 'lawn and garden', 'computing', 'electronics', 'art and craft', 'photography', 'storage', 'organization', 'pets', 'baby', 'kids', 'music', 'car and garage', 'toys', 'tools and hardware', 'jewelry and watches', 'sports', 'outdoor', 'books and magazines', 'health', 'beauty', 'fragrance', 'productivity', 'office supplies', 'holiday', 'deals', 'review']
len(candidate_labels)

34

In [16]:
samples = vd[:200]

In [17]:
desc = list(samples.Description.values)
titles = list(samples.Title.values)

In [18]:
results = classifier(titles, candidate_labels, multi_label=True)

In [24]:
title_results = results_to_df(results, candidate_labels, titles)

In [28]:
title_results[:5]

,travel,food,kitchen,home improvement,home decor,home furnishings,apparel,shoes,lawn and garden,computing,electronics,art and craft,photography,storage,organization,pets,baby,kids,music,car and garage,toys,tools and hardware,jewelry and watches,sports,outdoor,books and magazines,health,beauty,fragrance,productivity,office supplies,holiday,deals,review,classes,has_class,max_class,seq
0,0.000560,0.000129,0.001409,0.051707,0.072977,0.026421,0.000334,0.000159,0.000108,0.001424,0.001815,0.846226,0.000285,0.001996,0.016381,0.000182,0.006684,0.013627,0.000132,0.000077,0.000197,0.001514,0.000101,0.000218,0.092267,0.000064,0.009224,0.863963,0.000481,0.637818,0.000159,0.012571,0.254215,0.465044,[],False,None,TOP TEN LARGE WALL ART DIY IDEAS | DIY IKEA HACK | AFFORDABLE & AESTHETIC
1,0.000923,0.000465,0.154827,0.886514,0.995964,0.870846,0.000310,0.000078,0.000152,0.594050,0.000180,0.229701,0.000631,0.000736,0.075187,0.000332,0.004926,0.024010,0.000869,0.000036,0.000095,0.000215,0.000042,0.000301,0.048298,0.000178,0.008321,0.212648,0.000848,0.124867,0.000052,0.065048,0.827990,0.413366,[home decor],True,home decor,TOP 20 DIY DOLLAR TREE FALL HOME DECOR COMPILATION 2022 | HIGH END & NOT CHEESY FALL DECOR
2,0.000788,0.000410,0.505863,0.619941,0.447389,0.956606,0.000778,0.000243,0.026142,0.018168,0.003151,0.000721,0.000677,0.171187,0.413504,0.000647,0.027197,0.019266,0.000276,0.000190,0.000416,0.010754,0.000071,0.000277,0.056534,0.000044,0.035246,0.542973,0.001604,0.409840,0.000832,0.009865,0.254455,0.173438,[home furnishings],True,home furnishings,"DIY IKEA HACKS | AFFORDABLE, AESTHETIC, AND SUPER EASY 2022"
3,0.000171,0.000134,0.163792,0.989792,0.997865,0.995192,0.000317,0.000112,0.000074,0.001345,0.000295,0.011346,0.000161,0.000653,0.043383,0.000176,0.007868,0.001562,0.000232,0.000066,0.000101,0.000115,0.001174,0.000125,0.003289,0.000078,0.000650,0.752790,0.001029,0.078293,0.000039,0.035697,0.752750,0.196568,"[home decor, home furnishings, home improvement]",True,home decor,THRIFT WITH ME FOR HIGH END HOME DECOR | DIY THRIFT FLIP ROOM DECOR + HUGE DECOR THRIFT HAUL
4,0.019480,0.000743,0.402995,0.671087,0.993471,0.950770,0.005256,0.002019,0.166700,0.081726,0.002620,0.392948,0.001423,0.018240,0.544463,0.001646,0.030374,0.028860,0.001503,0.006604,0.001398,0.000739,0.001105,0.002265,0.327058,0.000472,0.108983,0.558210,0.057471,0.260906,0.000255,0.087008,0.482770,0.495134,"[home decor, home furnishings]",True,home decor,TOP 22 THRIFTED HOME DECOR FINDS | DIY THRIFTED HOME DECOR COMPILATION


In [21]:
title_results.has_class.describe()

count      200
unique       2
top       True
freq       152
Name: has_class, dtype: object

In [30]:
freq_dist(title_results, 'max_class')

,cnt,percent
food,26,13.0
home improvement,24,12.0
beauty,11,5.5
home decor,11,5.5
organization,10,5.0
art and craft,9,4.5
review,8,4.0
travel,7,3.5
outdoor,6,3.0
electronics,6,3.0


In [31]:
title_results.groupby(by='max_class', group_keys=True).apply(lambda df: df['seq'][:10])

max_class            
apparel           16                                                            HUGE Trendy Shein Summer Haul 2022!! *try on*
                  29                                                                  FASHION HAUL | Current Favorite Outfits
                  103                                                   Dark Academia capsule wardrobe 🤎 18 items, 45 outfits
                  175                                                                              Shiny Fashion [ASOS] P. 99
art and craft     5                        DECORATE WITH ME FOR FALL | DIY THRIFT FLIPS + HUGE FALL INSPIRED THRIFT HAUL 2022
                  39                               Gunfighter! - Jessie James Wallet: Leather Origami! (16 Rivets & NO Glue!)
                  54            How to Create a Gorgeous Resin Painting!! ~ Fuchsia, Purple and Turquoise! ~ "Pink Lightning"
                  75                                                   TUFTED SKELETON HAND MIRR

In [31]:
desc_results = results_to_df(classifier(desc, candidate_labels, multi_label=True), candidate_labels, desc)

In [32]:
desc_results

,travel,food and kitchen,home improvement,clothing and shoes,gardening,seq
0,0.834301,0.771446,0.954429,0.501327,0.689321,"+ Make sure to LIKE & SUBSCRIBE if you ejoyed this week's video! 💛 + Comment below to let me know what your favorite DIY was! 💛 + Follow me on Instagram @fiagarciadiy for more DIYs & behind the scenes content! 💛 @Emily Faith https://www.youtube.com/channel/UCv_pYa5knpzEg1Nk67JS8kA DIY COMPILATION ALERT! And this time the focus is ALL ABOUT LARGE ART WORK IDEAS ON A BUDGET. Buying artwork for your home can often times be one of the MOST expensive things you can purchase. So why not DIY some instead! In this video, you will see 10 aesthetic and affordable LARGE ART WORK ideas no one would ever guess you made. DIY, don't buy! Right? I hope you leave today feeling INSPIRED and ENCOURAGED to try something you've never done before. Xo, Fia #DIY #IKEAHACKS #IKEA ____________________________________________ + FIND ME HERE TOO: Instagram: fiagarciadiy BUSINESS INQUIRIES: Agarcia8@kent.edu ____________________________________________ + SHOP THIS VIDEO Latch: https://amzn.to/3N0A96p D rings: https://amzn.to/3xTW3nA Fabric scissors: https://amzn.to/3MV0vH8 Wall sconces: https://amzn.to/3xRfL3x Olive tree: https://amzn.to/3zND53k Faux Wood Peel & Stick Contact Paper: https://amzn.to/2YIgHFX 2pc Leather Drawer Pulls / knobs: https://amzn.to/3898M7u Circular Wooden Platter: https://amzn.to/3gilDr5 Rust-oleumMatte Black Spray Paint: https://amzn.to/3icKbTO Krylon Gold Spray Paint: https://amzn.to/31lvPux Pillow Inserts: https://amzn.to/2VrAOpB Velvet Pillow Covers (Orange/Red): https://amzn.to/3eImWyP Mid-Century Mondern furniture legs: https://amzn.to/3kPIahO Macrame Spool: https://amzn.to/2EkJqJ9 Office rug: https://amzn.to/3cYXt2K Faux plant: https://amzn.to/2UI3lqB Hanging indoor pots: https://amzn.to/3htFwfV Faux leather tape (Whiskey brown): https://amzn.to/3iWr67G Light Fixture Sconces: https://amzn.to/362GhHt **Amazon Affiliate Links** + EQUIPMENT USED Canon 80D DSLR with EF-S 18-55mm STM Lens: https://amzn.to/3eJT31o Neewer Ring Light Kit: https://amzn.to/3897MAg Blue Yeti USB Microphone: https://amzn.to/3g3MvL8 Canon EF 50mm f/1.4 USM Lens: https://amzn.to/3g4v257 Joby GorillaPod Flexible Compact Tripod: https://amzn.to/2BO01DE Sandisk 128GB Extreme Pro SD Card: https://amzn.to/31nQo9C **Amazon Affiliate Links** ____________________________________________ +MUSIC +Vexento - With You (Vlog No Copyright Music): https://youtu.be/wTD1KP8AyUE +LAKEY INSPIRED - Better Days: https://www.youtube.com/watch?v=RXLzv... +LAKEY INSPIRED - The Process +LAKEY INSPIRED - Chill Day ____________________________________________ + CHECK OUT MY OTHER VIDEOS! LIVING ROOM/OFFICE MAKEOVER | 5 HOME DECOR DIYS + 10 BABY PROOFING HACKS YOU NEED NOW!: https://youtu.be/VcL7EsFux1Y DIY COASTAL ROOM DECOR | DIY DOLLAR TREE SUMMER HOME DECOR: https://youtu.be/nHXgHoMA_h0 DIY HOME DECOR FOR FREE | 5 MINIMALIST DIY ROOM MAKEOVER HACKS: https://youtu.be/wb6M8OvM3pY DIY BOHO ROOM DECOR ON A BUDGET | BOHEMIAN LIVING ROOM HOME DECOR: https://youtu.be/oxaM5sYebSM DIY HOME DECOR ON A BUDGET | TRASH TO TREASURE DIY SHELF DECOR FOR FREE: https://youtu.be/O_JXlm_pQ0E DIY BACKYARD PATIO DECOR AND MAKEOVER ON A BUDGET | HOW TO MAKE A DIY RAISED PLANTER BOX!: https://youtu.be/N_sXFuZxe5c DIY ROOM MAKEOVER FOR FREE | 3 DIY HOME DECOR HACKS DURING QUARANTINE: https://youtu.be/bswTXiEhuWc DIY ROOM MAKEOVER FOR FREE | 6 DIY HOME DECOR HACKS DURING QUARANTINE: https://youtu.be/Mmj5DcLqReg"
1,0.103337,0.024301,0.956403,0.003956,0.023772,"+ Make sure to LIKE & SUBSCRIBE if you liked this week's video! + Comment below to let me know what your favorite DIY was! + Follow me on Instagram @fiagarciadiy for more DIYs & behind the scenes content! DIY MARATHON ALERT!!! This week, I went to Dollar Tree ready to fully start decorating for Fall on a budget! Watch to see how I upcycle some of these basic items and transform them into something that looks like you might be abl

In [33]:
title_results.apply(lambda s: (s > 0.9).index.values, axis=1)

TypeError: '>' not supported between instances of 'str' and 'float'